In [62]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [63]:
data = pd.read_csv('C:\\AIdata\\diabetes.csv')

In [64]:
features = data.drop(columns=['Outcome'])

In [65]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [66]:
X_train, X_test = train_test_split(features_scaled, test_size=0.2, random_state=42)

In [67]:
input_dim = X_train.shape[1]
latent_dim = 2
intermediate_dim = 64

In [68]:
inputs = Input(shape=(input_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [69]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [70]:
z = Lambda(sampling)([z_mean, z_log_var])

In [71]:
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [72]:
vae = Model(inputs, x_decoded_mean)

In [73]:
reconstruction_loss = Lambda(lambda x: K.sum(K.square(x[0] - x[1]), axis=-1), 
                             output_shape=(1,))([inputs, x_decoded_mean])
kl_loss = Lambda(lambda x: -0.5 * K.sum(1 + x[1] - K.square(x[0]) - K.exp(x[1]), axis=-1), 
                 output_shape=(1,))([z_mean, z_log_var])

In [74]:
vae.add_loss(K.mean(reconstruction_loss + kl_loss))

AttributeError: 'str' object has no attribute 'base_dtype'

In [ ]:
vae.compile(optimizer='adam')

In [ ]:
vae.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

In [ ]:
encoder = Model(inputs, z_mean)
z_test = encoder.predict(X_test)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(z_test[:, 0], z_test[:, 1], c=data['Outcome'].values[:len(z_test)], cmap='viridis')
plt.colorbar()
plt.xlabel("z[0]")
plt.ylabel("z[1]")
plt.show()